tgb - 01/18/2022 - Comparing four ways of predicting precip in an Earth-like setting:  
1. Brute force   
2. Brute force, but causally informed   
3. Climate invariant     
4. Climate invariant, but causally informed  

# Imports

In [8]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.set_memory_growth(physical_devices[1], True)
tf.config.experimental.set_memory_growth(physical_devices[2], True)
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

from cbrain.climate_invariant import *

import pickle

# BF

## Data generator

In [9]:
path_data = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/SPCAM_PHYS/'

In [55]:
in_vars = ['QBP','TBP','PS','SOLIN','SHFLX','LHFLX','PRECTt-dt']
out_vars = ['PRECT']
path_train = path_data + '2022_01_17_RG_TRAIN_M4K_PRECTt-dt.nc'
path_valid = path_data + '2022_01_17_RG_VALID_M4K_PRECTt-dt.nc'
path_norm = path_data + '2022_01_13_NORM_For_Nando_CI_t-dt.nc'

In [23]:
scale_dict = pickle.load(open(path_data+'CIML_Zenodo/009_Wm2_scaling.pkl','rb'))

In [24]:
N_batch = 8192

In [25]:
train_gen_BF = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

In [26]:
valid_gen_BF = DataGeneratorCI(
    data_fn = path_valid,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

## NN architecture

In [27]:
inp = Input(shape=(65,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(1, activation='relu')(densout)
model = tf.keras.models.Model(inp, dense_out)

In [28]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 65)]              0         
_________________________________________________________________
dense_16 (Dense)             (None, 128)               8448      
_________________________________________________________________
leaky_re_lu_14 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_15 (LeakyReLU)   (None, 128)               0         
_________________________________________________________________
dense_18 (Dense)             (None, 128)               16512     
_________________________________________________________________
leaky_re_lu_16 (LeakyReLU)   (None, 128)               0   

In [29]:
model.compile(tf.keras.optimizers.Adam(), loss=mse)

## Training

In [30]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2022_01_18_BF_PRECTt-dt'

In [31]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [1]:
Nep = 20
model.fit_generator(train_gen_BF, epochs=Nep, validation_data=valid_gen_BF,\
                    callbacks=[earlyStopping, mcp_save_pos])

NameError: name 'model' is not defined

# Climate-invariant

## Data Generator

In [56]:
path_norm_RH = path_data + '2021_02_01_NORM_O3_RH_small.nc'
path_norm_BMSE = path_data + '2021_06_16_NORM_BMSE_small.nc'
path_norm_LHF_nsDELQ = path_data + '2021_02_01_NORM_O3_LHF_nsDELQ_small.nc'

In [57]:
def train_gen_rescaling(input_rescaling):
    return DataGeneratorCI(
        data_fn = path_train,
        input_vars = input_rescaling,
        output_vars = out_vars,
        norm_fn = path_norm,
        input_transform = ('mean', 'maxrs'),
        output_transform = scale_dict)

In [58]:
train_gen_RH = train_gen_rescaling(in_vars)
train_gen_BMSE = train_gen_rescaling(in_vars)
train_gen_LHF_nsDELQ = train_gen_rescaling(in_vars)

In [59]:
train_gen_CI = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_norm,
    input_transform = ('mean','maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch,
    Qscaling = 'RH',
    Tscaling = 'BMSE',
    LHFscaling = 'LHF_nsDELQ',
    hyam=hyam, hybm=hybm, # Arrays to define mid-levels of hybrid vertical coordinate
    inp_sub_Qscaling=train_gen_RH.input_transform.sub, # What to subtract from RH inputs
    inp_div_Qscaling=train_gen_RH.input_transform.div, # What to divide RH inputs by
    inp_sub_Tscaling=train_gen_BMSE.input_transform.sub,
    inp_div_Tscaling=train_gen_BMSE.input_transform.div,
    inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
    inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div
)

In [60]:
valid_gen_CI = DataGeneratorCI(
    data_fn = path_valid,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_norm,
    input_transform = ('mean','maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch,
    Qscaling = 'RH',
    Tscaling = 'BMSE',
    LHFscaling = 'LHF_nsDELQ',
    hyam=hyam, hybm=hybm, # Arrays to define mid-levels of hybrid vertical coordinate
    inp_sub_Qscaling=train_gen_RH.input_transform.sub, # What to subtract from RH inputs
    inp_div_Qscaling=train_gen_RH.input_transform.div, # What to divide RH inputs by
    inp_sub_Tscaling=train_gen_BMSE.input_transform.sub,
    inp_div_Tscaling=train_gen_BMSE.input_transform.div,
    inp_sub_LHFscaling=train_gen_LHF_nsDELQ.input_transform.sub,
    inp_div_LHFscaling=train_gen_LHF_nsDELQ.input_transform.div
)

In [68]:
path_train = path_data + '2022_01_18_TRAIN_CI_PRECTt-dt.nc'
path_valid = path_train

In [69]:
in_vars = ['RH','BMSE','PS','SOLIN','SHFLX','LHF_nsDELQ','PRECTt-dt']
out_vars = ['PRECT']

In [70]:
train_gen_CLINV = DataGeneratorCI(
    data_fn = path_train,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

valid_gen_CLINV = DataGeneratorCI(
    data_fn = path_valid,
    input_vars = in_vars,
    output_vars = out_vars,
    norm_fn = path_norm,
    input_transform = ('mean', 'maxrs'),
    output_transform = scale_dict,
    shuffle = False,
    batch_size=N_batch
)

## NN architecture

In [61]:
inp = Input(shape=(65,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(1, activation='relu')(densout)
model2 = tf.keras.models.Model(inp, dense_out)

In [71]:
inp = Input(shape=(65,)) ## input after rh and tns transformation
densout = Dense(128, activation='linear')(inp)
densout = LeakyReLU(alpha=0.3)(densout)
for i in range (6):
    densout = Dense(128, activation='linear')(densout)
    densout = LeakyReLU(alpha=0.3)(densout)
dense_out = Dense(1, activation='relu')(densout)
model3 = tf.keras.models.Model(inp, dense_out)

## Training

In [62]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2022_01_18_CI_PRECTt-dt'

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [63]:
model2.compile(tf.keras.optimizers.Adam(), loss=mse)

In [64]:
Nep = 20
model2.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos])

Epoch 1/20
17475/17475 [==============================] - 4111s 235ms/step - loss: 31013.5917 - val_loss: 36151.2643
Epoch 2/20
17475/17475 [==============================] - 4403s 252ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 3/20
17475/17475 [==============================] - 4104s 235ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 4/20
17475/17475 [==============================] - 4102s 235ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 5/20
17475/17475 [==============================] - 4590s 263ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 6/20
17475/17475 [==============================] - 4327s 248ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 7/20
17475/17475 [==============================] - 4169s 239ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 8/20
 8535/17475 [=============>................] - ETA: 19:26 - loss: 37333.9169

KeyboardInterrupt: 

In [72]:
# Where to save the model
path_HDF5 = '/DFS-L/DATA/pritchard/tbeucler/SPCAM/HDF5_DATA/'
save_name = '2022_01_19_CLINV_PRECTt-dt'

earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=0, mode='min')
mcp_save_pos = ModelCheckpoint(path_HDF5+save_name+'.hdf5',save_best_only=True, monitor='val_loss', mode='min')

In [73]:
model3.compile(tf.keras.optimizers.Adam(), loss=mse)

In [74]:
Nep = 20
model3.fit_generator(train_gen_CI, epochs=Nep, validation_data=valid_gen_CI,\
                    callbacks=[earlyStopping, mcp_save_pos])

Epoch 1/20
17475/17475 [==============================] - 4351s 249ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 2/20
17475/17475 [==============================] - 4081s 234ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 3/20
17475/17475 [==============================] - 4128s 236ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 4/20
17475/17475 [==============================] - 4375s 250ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 5/20
17475/17475 [==============================] - 4074s 233ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 6/20
17475/17475 [==============================] - 4070s 233ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 7/20
17475/17475 [==============================] - 4063s 233ms/step - loss: 37279.9563 - val_loss: 36151.2643
Epoch 8/20
17474/17475 [============================>.] - ETA: 0s - loss: 37278.4385

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-74-381fc6301eb7>", line 3, in <module>
    callbacks=[earlyStopping, mcp_save_pos])
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training.py", line 1297, in fit_generator
    steps_name='steps_per_epoch')
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/tensorflow_core/python/keras/engine/training_generator.py", line 323, in model_iteration
    steps_name='validation_steps')
  File "/DFS-L/DATA/pritchard/tbeucler/Miniconda3_2021_04_12/miniconda3/envs/CbrainCustomLayer/lib/python3.6/site-packages/tensorflow_core/py

KeyboardInterrupt: 